In [1]:
import pickle
from datasets import load_dataset
import numpy as np


multi_lexsum = load_dataset("allenai/multi_lexsum", name="v20230518")
modified_dataset = multi_lexsum["test"].filter(lambda x: x["summary/short"] != None)

In [2]:
import evaluate

rouge_scoring = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

In [4]:
p = "Plaintiffs Mitchell and Williams were unlawfully imprisoned by the City for failure to pay debts and coerced to perform labor in the City jail under poor conditions to reduce their debts. The City violated their Fourteenth Amendment rights. The Court granted a preliminary injunction, ordering specific remedies and no monthly payments over $25 for indigent defendants; allowing community service instead of payment; altering Public Defenders' contracts; and prohibiting incarceration except under specific circumstances. The City officials involved were required to attend a hearing."

ex = modified_dataset["summary/short"][0]

print(rouge_scoring.compute(predictions=[p], references=[ex], use_stemmer = True))
print(bertscore.compute(predictions=[p], references=[ex], model_type="microsoft/deberta-large-mnli"))

# """
# {'rouge1': 0.55625, 'rouge2': 0.27044025157232704, 'rougeL': 0.28124999999999994, 'rougeLsum': 0.28124999999999994}
# {'precision': [0.70556640625], 'recall': [0.6686365604400635], 'f1': [0.6866052746772766], 'hashcode': 'microsoft/deberta-large-mnli_L18_no-idf_version=0.3.12(hug_trans=4.38.0.dev0)'}
# """

{'rouge1': 0.29015544041450775, 'rouge2': 0.07329842931937172, 'rougeL': 0.23834196891191708, 'rougeLsum': 0.23834196891191708}
{'precision': [0.5845822691917419], 'recall': [0.5562899112701416], 'f1': [0.5700852274894714], 'hashcode': 'microsoft/deberta-large-mnli_L18_no-idf_version=0.3.12(hug_trans=4.38.0.dev0)'}


In [6]:
print(ex)

The plaintiffs filed a lawsuit on March 8, 2014, alleging that the City of Montgomery, Alabama, improperly imprisoned them for failing to pay traffic fines. They alleged that they did not have an ability to pay the fines due to their financial circumstances and that the city did not consider their ability to pay. On May 1, 2014, the District Court granted the plaintiffs motion for a preliminary injunction, preventing the city from collecting more money from traffic tickets of plaintiffs'. On October 31, 2014 the parties filed to dismiss the case pursuant to a settlement agreement, which included numerous changes to Municipal Court proceedings. The case is now closed.


In [6]:
import os

## ROUGE
# ['led: rouge1: 48.77', 'led: rouge2: 25.24', 'led: rougeL: 33.67', 'led: rougeLsum: 33.94']
# ['primera: rouge1: 45.74', 'primera: rouge2: 22.83', 'primera: rougeL: 31.92', 'primera: rougeLsum: 32.18']

### BERTScore
# ['led: precision: 0.74', 'led: recall: 0.61', 'led: f1: 0.67']
# ['primera: precision: 0.74', 'primera: recall: 0.62', 'primera: f1: 0.68']

for file in os.listdir():
    if "151_predicted" in file:
        predicted = pickle.load(open(file, "rb"))
        print(file.split(".")[0])
        print([f"{metric} : {np.mean(val) * 100:0.2f}"for metric, val in rouge_scoring.compute(predictions=predicted, references=modified_dataset["summary/short"][:len(predicted)], use_stemmer = True).items()])
        print([f"{metric} : {np.mean(val[0]):0.2f}"for metric, val in bertscore.compute(predictions=predicted, references=modified_dataset["summary/short"][:len(predicted)], model_type="microsoft/deberta-large-mnli", batch_size = 2).items() if metric != "hashcode"])
        print("----------")


151_predicted_text_1shot_cot_summarization_random_selection_bert
['rouge1 : 30.18', 'rouge2 : 8.30', 'rougeL : 17.85', 'rougeLsum : 18.31']


['precision : 0.74', 'recall : 0.82', 'f1 : 0.78']
----------
151_predicted_text_1shot_cot_summarization_random_selection
['rouge1 : 31.98', 'rouge2 : 8.59', 'rougeL : 18.47', 'rougeLsum : 18.95']


['precision : 0.69', 'recall : 0.78', 'f1 : 0.73']
----------
151_predicted_basic_text_1shot_cot_summarization_random_selection
['rouge1 : 31.06', 'rouge2 : 7.45', 'rougeL : 18.27', 'rougeLsum : 18.80']
['precision : 0.81', 'recall : 0.89', 'f1 : 0.85']
----------
151_predicted_basic_text_1shot_cot_summarization_random_selection_bert
['rouge1 : 30.45', 'rouge2 : 7.35', 'rougeL : 18.34', 'rougeLsum : 18.60']
['precision : 1.00', 'recall : 1.00', 'f1 : 1.00']
----------


In [8]:
predicted = pickle.load(open("151_predicted_basic_text_1shot_cot_summarization_random_selection_bert.pickle", "rb"))

In [11]:
modified_dataset["summary/short"][1]

"In 2013, indigent detainee filed a complaint against the City of Montgomery and a Municipal Court Judge for unconstitutionally ordering the petitioner to serve time for her inability to pay court-order fines and fees for their traffic violations. The complaint was originally filed in the Circuit Court of Montgomery County, Alabama and then transferred to the U.S. District Court for the Middle District of Alabama, where it was consolidated with a similar case. In the amended complaint, petitioners alleged that the imprisonment orders violated their Sixth and Fourteenth Amendment Rights. Petitioners sought remedy through injunctive and declaratory relief. In 2014, the parties reached a settlement that provided new judicial procedures for the Municipal Court to follow regarding indigent defendants and nonpayment. It also included three declarations, most importantly, it declared that the constitutional principles set out in Bearden v. Georgia, 461 U.S. 660 (1983), and Turner v. Rogers, 1

In [16]:
predicted[1]

"The plaintiff, Terrick Terrell Nooner, filed a lawsuit alleging that the policies and practices of the Arkansas Department of Correction (ADC) violated his civil rights and constitutional rights. Nooner is currently under the supervision of the ADC and awaiting execution of a death sentence. The defendants argue that Nooner did not exhaust all administrative remedies before bringing the lawsuit. The court denied the defendants' motion to dismiss the case. Nooner also sought intervention in the case by another individual, Davis, who is scheduled to be executed. The court denied Davis's request to intervene. The court also denied Nooner's request for a preliminary injunction. Nooner appealed the court's decision. The case is ongoing."